CONTENT WARNING: explicit and multiple references to suicide attemps and suicidal ideation

The following is an analysis regarding attempted suicides and suicidal thoughts. Questions around these topics will be directly referenced and mentioned explicitly. While their potential correlations with other parameters will be analysed and discussed at length, suicide attempts and suicidal ideation as concepts will not be discussed. Deaths by suicide will not be referenced or discussed at all.

Please only continue if it comfortable for you to do so.

In [12]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.cross_validation import train_test_split
import seaborn as sb
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle

2016 results ( https://www.datafiles.samhsa.gov/study-dataset/national-survey-drug-use-and-health-2016-nsduh-2016-ds0001-nid17185 )

In [6]:
df = pd.DataFrame.from_csv('NSDUH_2016_Tab.tsv',sep='\t')

/anaconda3/envs/condasc/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [7]:
df

,FILEDATE,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,...,POVERTY3,TOOLONG,TROUBUND,PDEN10,COUTYP4,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP
QUESTID2,,,,,,,,,,,,,,,,,,,,,
11635143,02/28/2018,1,99,99,16,9999,99,4,93,93,...,3.0,2,2,3,3,2,2,819.434247,40037,2
36845143,02/28/2018,1,99,99,15,9999,99,1,7,99,...,3.0,1,2,2,2,2,2,280.624352,40013,2
35755143,02/28/2018,1,99,99,26,9999,99,1,7,99,...,2.0,2,2,1,1,2,2,10133.833583,40036,2
94475143,02/28/2018,2,4,4,991,9991,91,91,91,91,...,3.0,2,2,1,1,2,2,2284.717175,40028,1
92675143,02/28/2018,1,99,99,5,9999,99,4,93,93,...,3.0,2,2,1,1,2,2,24815.892373,40009,2
59596143,02/28/2018,2,99,99,991,9991,91,91,91,91,...,2.0,1,2,2,2,2,2,5546.690814,40016,1
41106143,02/28/2018,2,99,99,991,9991,91,91,91,91,...,3.0,1,2,2,2,2,2,9539.366625,40002,1
96416143,02/28/2018,2,99,99,991,9991,91,91,91,91,...,3.0,2,2,2,2,2,2,4144.946988,40021,2
73716143,02/28/2018,1,99,99,15,9999,99,1,30,99,...,1.0,2,2,2,2,2,2,2940.093267,40009,2


In [11]:
print('Num adults w/ suicidal thoughts: '+str(len(df[(df.MHSUITHK==1) | (df.ADWRSTHK==1)])))
print('Num adults w/ suicidal plans: '+str(len(df[(df.MHSUIPLN==1) | (df.ADWRSPLN==1)])))
print('Num adults w/ suicidal attempt: '+str(len(df[(df.MHSUITRY==1) | (df.ADWRSATP==1)])))
print('Num adults w/ thoughts of death: '+str(len(df[(df.ADWRDLOT==1) | (df.ADWRDBTR==1)])))
print('Num adults w/ thoughts of death or suicide: '+str(len(df[(df.ADWRDLOT==1) | (df.ADWRDBTR==1) | (df.MHSUITHK==1) | (df.ADWRSTHK==1)])))

Num adults w/ suicidal thoughts: 3930
Num adults w/ suicidal plans: 1444
Num adults w/ suicidal attempt: 958
Num adults w/ thoughts of death: 4612
Num adults w/ thoughts of death or suicide: 5598


In [22]:
results17 = pickle.load(open('logit_2017data','rb'))

In [27]:
def prepData17(df,use_col,use_param):
    rem_val = [81,83,85,89,91,93,94,97,98,99,981,983,985,989,991,993,994,997,998,999,9981,9983,9985,9989,9991,9993,
               9994,9997,9998,9999,186,187,188,180]

    dfc = df.copy()
    dfc = dfc[(dfc.MHSUITHK==0) | (dfc.MHSUITHK==1)]
    
    dfc.HTINCHE2 = np.floor((dfc.HTINCHE2-55)/5)
    dfc.WTPOUND2 = np.floor((dfc.WTPOUND2-75)/55)
    dfc.BMI2 = np.floor((dfc.BMI2-9.3)/14.83)
    dfc.EDUSCHLGO.replace(11,1,inplace=True)
    dfc.BOOKED.replace(3,1,inplace=True)
    dfc.SNYSELL.replace(1,0,inplace=True)
    dfc.SNYSTOLE.replace(1,0,inplace=True)
    dfc.SNYATTAK.replace(1,0,inplace=True)
    dfc.SNYSELL.replace([2,3,4,5],1,inplace=True)
    dfc.SNYSTOLE.replace([2,3,4,5],1,inplace=True)
    dfc.SNYATTAK.replace([2,3,4,5],1,inplace=True)
    
    y = dfc.MHSUITHK
    use_X = dfc[use_col]
    dum_p = []
    
    for param in use_col:
        if use_X[param].max() == 2:
            use_X[param].replace(2,0,inplace=True)
            X_teu[param].replace(2,0,inplace=True)
        elif use_X[param].max() > 2:
            dum_p.append(param)
    
    use_X = pd.get_dummies(use_X,columns=dum_p) 
    return use_X[use_param],y
    

In [36]:
use_col17 = ['CATAG6','HEALTH','MOVSINPYR2','SEXATRACT','IRMARIT','IREDUHIGHST2','NEWRACE2','WRKSTATWK2','IRPINC3',
           'SNRLGIMP','WTPOUND2','BMI2']
use_param17 = ['CATAG6_2', 'CATAG6_3', 'CATAG6_5', 'CATAG6_6', 'HEALTH_1','HEALTH_2', 'HEALTH_4', 'HEALTH_5',
               'MOVSINPYR2_2','MOVSINPYR2_3', 'SEXATRACT_1', 'SEXATRACT_3', 'IRMARIT_1', 'IRMARIT_2','IREDUHIGHST2_6',
               'IREDUHIGHST2_9', 'NEWRACE2_2','NEWRACE2_5', 'NEWRACE2_7','IRPINC3_5', 'IRPINC3_7', 'SNRLGIMP_1']
X16,y16 = prepData17(df,use_col17,use_param17)

In [37]:
y_pred = np.round(results17.predict(X16))
print('fraction wrong: '+str(np.sum(np.abs(y_pred-y16))/len(y16)))
confusion_matrix(y16, y_pred)

fraction wrong: 0.291016917116


array([[28503, 11423],
       [  894,  1504]])

Type 1 error: 11423 (False positive (suicidal ideation in this case))

Type 2 error: 894 (False negative (did not predict SI))

In [39]:
print(classification_report(y16, y_pred))

             precision    recall  f1-score   support

        0.0       0.97      0.71      0.82     39926
        1.0       0.12      0.63      0.20      2398

avg / total       0.92      0.71      0.79     42324



In [43]:
len(df[df.MHSUITHK==1]),len(df[df.MHSUITHK==0])

(2398, 39926)

Model using 2017 data (used undersampling) does not perform well with 2016 data.

Next step: explore with more data and try other methods for dealing with unbalanced sample